In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 1000

In [2]:
df = pd.read_csv('all_merged.csv', sep=",")

In [3]:
df

,Unnamed: 0,title,review,rating
0,0,"Occasionally awkward, but very good overall",Rika Nishina (Megumi Okina) works for a social...,9/10
1,1,"Non-Linear, yet Effective and Creepy.","Although the word ""grudge"" doesn't quite fit t...",10/10
2,2,Pretty Good,"""Ju-On, The Grudge"" is not an easy movie to fi...",7/10
3,3,Very Scary Japanese Horror Movie,"In Japan, when the volunteer social assistant ...",8/10
4,4,A genuinely creepy Japanese ghost story master...,Following in the immensely successful footstep...,10/10
...,...,...,...,...
4782,413,Japanese Horror Remakes... from this author it...,"Now having seen both The Ring and Dark Water, ...",4/10
4783,414,I might have gotten startled ONCE during this ...,"THe movie is slow, grinding, and uneventful. A...",4/10
4784,415,That Movie Blew Harder than Yeltsin into a Bre...,I've never written comments on a movie before....,3/10
4785,416,"The ""Dark Water"" is really dark, for a reason.","I'm 47, married 26 years, four kids. I was rai...",1/10


In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df

,title,review,rating
0,"Occasionally awkward, but very good overall",Rika Nishina (Megumi Okina) works for a social...,9/10
1,"Non-Linear, yet Effective and Creepy.","Although the word ""grudge"" doesn't quite fit t...",10/10
2,Pretty Good,"""Ju-On, The Grudge"" is not an easy movie to fi...",7/10
3,Very Scary Japanese Horror Movie,"In Japan, when the volunteer social assistant ...",8/10
4,A genuinely creepy Japanese ghost story master...,Following in the immensely successful footstep...,10/10
...,...,...,...
4782,Japanese Horror Remakes... from this author it...,"Now having seen both The Ring and Dark Water, ...",4/10
4783,I might have gotten startled ONCE during this ...,"THe movie is slow, grinding, and uneventful. A...",4/10
4784,That Movie Blew Harder than Yeltsin into a Bre...,I've never written comments on a movie before....,3/10
4785,"The ""Dark Water"" is really dark, for a reason.","I'm 47, married 26 years, four kids. I was rai...",1/10


In [22]:
df['review'][88]

'The "spirit" in this movie was kind of creepy, but I felt like I wasted an afternoon watching this piece of pooh. It didn\'t tie itself together very well, the acting was second rate and it was very slow. I thought there was a lot of potential with this, but they way it was done barely kept me awake. It would be very interesting to see someone take this story to another level. I\'m not sure what else to say about this, but I want my post to make it online now that I have invested the time to review this lame movie. Sorry to be dragging on... but 10 lines of text are required. We\'re almost there... Hang on... Wow! I feel like I\'m creating a movie like the Grudge, now. Long, dull and a waste of time to deal with. Oh well, we\'re done now!'

In [6]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from tqdm import tqdm
from nltk import sent_tokenize,word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [7]:
main = []

# Storing all punctuations using RE library like !;,"% etc
re_puncs = re.compile('[%s]' % re.escape(string.punctuation))
# Storing all stop words like a, an, the, when, there, this etc
stop_word  = set(stopwords.words('english'))
stop_word.add("im")
# print(stop_word)


In [8]:
sentences_listed = [line.split(" ") for line in df['review']]

import gensim

word_model = gensim.models.Word2Vec(sentences = sentences_listed, vector_size = 100)
words = list(word_model.wv.key_to_index)
print(len(words))

11398


In [9]:
!pip install gensim

In [10]:
word_model.wv.most_similar("actress")

[('actress,', 0.9184087514877319),
 ('role', 0.8999068737030029),
 ('performance', 0.8671321868896484),
 ('Connelly', 0.8669130802154541),
 ('Watts', 0.8406627178192139),
 ('journalist.', 0.8305148482322693),
 ('Bell', 0.823521614074707),
 ('actor', 0.8173946142196655),
 ('lead,', 0.8150674104690552),
 ('Browning', 0.8133652210235596)]

In [11]:
word_model.wv.most_similar("death")

[('body', 0.887908935546875),
 ('spirit', 0.8851573467254639),
 ('family', 0.8849808573722839),
 ('living', 0.8844324350357056),
 ('curse', 0.8831897974014282),
 ('visions', 0.8733584880828857),
 ('house,', 0.8723907470703125),
 ('house', 0.868007242679596),
 ('closet', 0.866958498954773),
 ('computer', 0.8587065935134888)]

In [12]:
word_model.wv.most_similar("scary")

[('bad', 0.8312276005744934),
 ('scary,', 0.7627571225166321),
 ('good', 0.7607622146606445),
 ('boring', 0.6972141861915588),
 ('scary.', 0.6787448525428772),
 ('fun', 0.6403161883354187),
 ('frightening', 0.6154860854148865),
 ('stupid', 0.6127957105636597),
 ('really', 0.6106213927268982),
 ('just', 0.6083388328552246)]

In [13]:
# Making Lemmatizing object
lem = WordNetLemmatizer()
# Using Porter Stemmer
p_stem = PorterStemmer()

# Traversing whole dataset
for i in tqdm(range(len(df['review']))):
    # Tokenization
    tokens = word_tokenize(str(df['review'][i]))
    # Converting all characters to lower case
    tokens = [w.lower() for w in tokens]
    # Remove all punctuations from sentenses
    tokens = [re_puncs.sub('', w) for w in tokens]
    # Checking all words is alphabets or not
    tokens = [i for i in tokens if i.isalpha()]
    # Removing all stop words from the sentenses
    tokens = [w for w in tokens if w not in stop_word]
    # Doing Lemmatizing of words
    tokens = [lem.lemmatize(w) for w in tokens]
    # Stemming process
    tokens = [p_stem.stem(w) for w in tokens]
    # Finally convert to string
    r = ' '.join(tokens)
    # Storing the final string into main list
    main.append(r)


100%|██████████| 4787/4787 [00:39<00:00, 121.25it/s]


In [14]:
df['translated'] = main

In [15]:
df

,title,review,rating,translated
0,"Occasionally awkward, but very good overall",Rika Nishina (Megumi Okina) works for a social...,9/10,rika nishina megumi okina work social servic a...
1,"Non-Linear, yet Effective and Creepy.","Although the word ""grudge"" doesn't quite fit t...",10/10,although word grudg nt quit fit bill part titl...
2,Pretty Good,"""Ju-On, The Grudge"" is not an easy movie to fi...",7/10,juon grudg easi movi find america least nt fir...
3,Very Scary Japanese Horror Movie,"In Japan, when the volunteer social assistant ...",8/10,japan volunt social assist rika nishina megumi...
4,A genuinely creepy Japanese ghost story master...,Following in the immensely successful footstep...,10/10,follow immens success footstep ring juon grudg...
...,...,...,...,...
4782,Japanese Horror Remakes... from this author it...,"Now having seen both The Ring and Dark Water, ...",4/10,seen ring dark water begin wonder director com...
4783,I might have gotten startled ONCE during this ...,"THe movie is slow, grinding, and uneventful. A...",4/10,movi slow grind unev first minut see first sec...
4784,That Movie Blew Harder than Yeltsin into a Bre...,I've never written comments on a movie before....,3/10,never written comment movi never felt rob chea...
4785,"The ""Dark Water"" is really dark, for a reason.","I'm 47, married 26 years, four kids. I was rai...",1/10,marri year four kid rais watch televis movi sa...


In [16]:
#df['translated'][88]

'spirit movi kind creepi felt like wast afternoon watch piec pooh nt tie togeth well act second rate slow thought lot potenti way done bare kept awak would interest see someon take stori anoth level sure els say want post make onlin invest time review lame movi sorri drag line text requir almost hang wow feel like creat movi like grudg long dull wast time deal oh well done'

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['translated'])

In [18]:
print(len(tokenizer.word_counts))

22923


In [28]:
counter = 0
for key, value in tokenizer.word_counts.items():
    if value > 2400:
        print(key, value)
        counter+=1

print(counter)

one 4912
film 9819
realli 3083
nt 8349
watch 3920
good 3593
get 2556
stori 3056
end 2875
like 4527
see 3767
horror 6036
well 2449
make 3008
movi 13409
time 2782
scare 2522
scari 2650
ring 3209
go 2819
origin 2743
21


**grupa 1** scary/scared, afraid, fright(ened), alarmed, shocked, horror/horrified, threatening, hair-rising, horrendous

In [35]:
count1 = 0
num=tokenizer.word_counts['scari']
print(num)
count1+=num

2650


In [37]:
num=tokenizer.word_counts['scare']
print(num)
count1+=num

2522


In [40]:
num=tokenizer.word_counts['afraid']
print(num)
count1+=num

129


In [42]:
num=tokenizer.word_counts['fright']
print(num)
count1+=num

118


In [44]:
num=tokenizer.word_counts['frighten']
print(num)
count1+=num

518


In [47]:
num=tokenizer.word_counts['alarm']
print(num)
count1+=num

11


In [51]:
num=tokenizer.word_counts['shock']
print(num)
count1+=num

438


In [56]:
num=tokenizer.word_counts['horrifi']
print(num)
count1+=num

140


In [60]:
num=tokenizer.word_counts['horrif']
print(num)
count1+=num

94


In [61]:
num=tokenizer.word_counts['threaten']
print(num)
count1+=num

47


In [70]:
num=tokenizer.word_counts['horrend']
print(num)
count1+=num

15


In [74]:
print('WYNIK DLA GRUPY PIERWSZEJ TOOOOO',count1)

WYNIK DLA GRUPY PIERWSZEJ TOOOOO 6682


**grupa 2** fear, petrified, terror/terrified, distress, helpless(ness), anxiety/anxious, worried, apprehension, creepy, uneasy, disturbing, chilling, eerie, spooky(?), intimidating, overwhelming, unnerving

In [75]:
count2 = 0
num=tokenizer.word_counts['fear']
print(num)
count2+=num

468


In [77]:
num=tokenizer.word_counts['petrifi']
print(num)
count2+=num

8


In [79]:
num=tokenizer.word_counts['terror']
print(num)
count2+=num

267


In [80]:
num=tokenizer.word_counts['terrifi']
print(num)
count2+=num

421


In [81]:
num=tokenizer.word_counts['distress']
print(num)
count2+=num

18


In [82]:
num=tokenizer.word_counts['helpless']
print(num)
count2+=num

19


In [88]:
num=tokenizer.word_counts['anxious']
print(num)
count2+=num

2


In [89]:
num=tokenizer.word_counts['worri']
print(num)
count2+=num

61


In [92]:
num=tokenizer.word_counts['creepy']
print(num)
count2+=num

2


In [93]:
num=tokenizer.word_counts['creepi']
print(num)
count2+=num

1461


In [95]:
num=tokenizer.word_counts['uneasi']
print(num)
count2+=num

54


In [96]:
num=tokenizer.word_counts['disturb']
print(num)
count2+=num

453


In [99]:
num=tokenizer.word_counts['chilli']
print(num)
count2+=num

5


In [101]:
num=tokenizer.word_counts['eeri']
print(num)
count2+=num

276


In [105]:
num=tokenizer.word_counts['overwhelm']
print(num)
count2+=num

24


In [107]:
num=tokenizer.word_counts['overwhelming']
print(num)
count2+=num

1


In [112]:
print('WYNIK DLA GRUPY DRUGIEJ TOOOOO',count2)

WYNIK DLA GRUPY DRUGIEJ TOOOOO 3540
